This code uses part of:
* `How Private are DP-SGD Implementations?` [Colab notebook](https://colab.research.google.com/drive/1zI2H8YEXbQyD6gZVVskFwcOiM5YMvqRe?utm_source=catalyzex.com)

* `Hiding Among the Clones: A Simple and Nearly Optimal Analysis of Privacy Amplification by Shuffling`
[Github project](https://github.com/apple/ml-shuffling-amplification?utm_source=catalyzex.com)

In [1]:
!pip install dp_accounting
!pip install numba
# !pip install --upgrade numpy scipy numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 910.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# Imports

In [2]:
from typing import Dict, Any, List, Tuple, Callable #, Sequence
from dataclasses import dataclass
import inspect
import time
from functools import cache
import math
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numba import jit
from functools import lru_cache
from dp_accounting import pld, dp_event, rdp

ModuleNotFoundError: No module named 'numpy.char'

# Privacy bounds

### Local and Central

In [ ]:
# ================= Supporting function =================
def bin_search(func: Callable, lower: float, upper: float, target: float, increasing: bool = False) -> float:
    search_params = pld.common.BinarySearchParameters(lower, upper, (upper - lower) / 1_000)
    if increasing:
        return pld.common.monotone_function(func, target, search_params)
    return pld.common.inverse_monotone_function(lambda val: func(val), target, search_params)

# ==================== Deterministic ====================
@cache
def deterministic_delta(sigma: float,
                        epsilon: float,
                        ) -> float:
    upper_cdfs = stats.norm.cdf(0.5 / sigma - sigma * epsilon)
    lower_log_cdfs = stats.norm.logcdf(-0.5 / sigma - sigma * epsilon)
    return upper_cdfs - np.exp(epsilon + lower_log_cdfs)

@cache
def deterministic_epsilon(sigma: float,
                          delta: float,
                          epsilon_upper_bound: float = 100,
                          ) -> float:
    epsilon = bin_search(lambda eps: deterministic_delta(sigma=sigma, epsilon=eps),
                         0, epsilon_upper_bound, delta, increasing=False)
    return np.inf if epsilon is None else epsilon

# ==================== Local ====================
@cache
def local_delta(sigma: float,
                epsilon: float,
                num_selected: int,
                num_epochs: int,
                ) -> np.ndarray[float]:
    return deterministic_delta(sigma=sigma/np.sqrt(num_selected*num_epochs), epsilon=epsilon)

@cache
def local_epsilon(sigma: float,
                  delta: float,
                  num_selected: int,
                  num_epochs: int,
                  epsilon_upper_bound: float = 100,
                  ) -> float:
    return deterministic_epsilon(sigma=sigma/np.sqrt(num_selected*num_epochs), delta=delta, epsilon_upper_bound=epsilon_upper_bound)

### Poisson

In [ ]:
# ==================== PLD ====================
@cache
def poisson_pld(sigma: float,
                num_steps: int,
                num_epochs: int,
                sampling_prob: float,
                discretization: float = 1e-4,
                ) -> pld.privacy_loss_distribution:
    pl_dist = pld.privacy_loss_distribution.from_gaussian_mechanism(standard_deviation=sigma,
                                                                    pessimistic_estimate=True,
                                                                    value_discretization_interval=discretization,
                                                                    sampling_prob=sampling_prob,
                                                                    use_connect_dots=True)
    return pl_dist.self_compose(num_steps*num_epochs)

@cache
def poisson_delta_pld(sigma: float,
                      epsilon: float,
                      num_steps: int,
                      num_selected: int,
                      num_epochs: int,
                      sampling_prob: float = 0.0,
                      discretization: float = 1e-4,
                      ) -> float:
    if sampling_prob == 0.0:
        sampling_prob = num_selected/num_steps
    pld = poisson_pld(sigma=sigma, num_steps=num_steps, num_epochs=num_epochs, sampling_prob=sampling_prob,
                      discretization=discretization)
    return pld.get_delta_for_epsilon(epsilon)

@cache
def poisson_epsilon_pld(sigma: float,
                        delta: float,
                        num_steps: int,
                        num_selected: int,
                        num_epochs: int,
                        sampling_prob: float = 0.0,
                        discretization: float = 1e-4,
                        ) -> float:
    if sampling_prob == 0.0:
        sampling_prob = num_selected/num_steps
    pld = poisson_pld(sigma=sigma, num_steps=num_steps, num_epochs=num_epochs, sampling_prob=sampling_prob,
                      discretization=discretization)
    return pld.get_epsilon_for_delta(delta)

# ==================== RDP ====================
# @cache
def poisson_rdp(sigma: float,
                num_steps: int,
                num_epochs: int,
                sampling_prob: float,
                alpha_orders: List[float],
                ) -> rdp.RdpAccountant:
    accountant = rdp.RdpAccountant(alpha_orders)
    event = dp_event.PoissonSampledDpEvent(sampling_prob, dp_event.GaussianDpEvent(sigma))
    accountant.compose(event, int(num_steps*num_epochs))
    return accountant

# @cache
def poisson_delta_rdp(sigma: float,
                      epsilon: float,
                      num_steps: int,
                      num_selected: int,
                      num_epochs: int,
                      sampling_prob: float = 0.0,
                      alpha_orders: List[float] = None,
                      print_alpha: bool = False,
                      ) -> float:
    if sampling_prob == 0.0:
        sampling_prob = num_selected/num_steps
    accountant = poisson_rdp(sigma=sigma, num_steps=num_steps, num_epochs=num_epochs, sampling_prob=sampling_prob,
                             alpha_orders=alpha_orders)
    delta, used_alpha = accountant.get_delta_and_optimal_order(epsilon)
    if print_alpha:
        print(f'Used alpha: {used_alpha}')
        return delta
    return accountant.get_delta(epsilon)

# @cache
def poisson_epsilon_rdp(sigma: float,
                        delta: float,
                        num_steps: int,
                        num_selected: int,
                        num_epochs: int,
                        sampling_prob: float = 0.0,
                        alpha_orders: List[float] = None,
                        print_alpha: bool = False,
                        ) -> float:
    if sampling_prob == 0.0:
        sampling_prob = num_selected/num_steps
    accountant = poisson_rdp(sigma=sigma, num_steps=num_steps, num_epochs=num_epochs, sampling_prob=sampling_prob,
                             alpha_orders=alpha_orders)
    epsilon, used_alpha = accountant.get_epsilon_and_optimal_order(delta)
    if print_alpha:
        print(f'Used alpha: {used_alpha}')
        return epsilon
    return accountant.get_epsilon(delta)

### Shuffle

In [ ]:
# For licensing see accompanying LICENSE file.
# Copyright (C) 2020 Apple Inc. All Rights Reserved.


# This document contains 4 computations: 2 empirical and 2 theoretical.
# 1. Empirical analysis
# 2. Theoretical analysis

# ========= SUPPORT FUNCTIONS ==========

# This function uses binary search to approximate the smallest epsilon such that deltacomp will output something smaller than delta (i.e. an algorithm is (epsilon, delta)-DP)
def binarysearch(deltacomp, delta, num_iterations, epsupper):
    '''
    binary search to find min epsilon such that deltacomp(epsilon)<delta
    deltacomp = function that takes epsilon as input and outputs delta
    num_iterations = number of iterations, accuracy is 2^(-num_iterations)*epsupper
    epsupper = upper bound for epsilon. You should be sure that deltacomp(epsupper)<delta.
    '''
    llim = 0
    rlim = epsupper
    for t in range(num_iterations):
        mideps = (rlim + llim) / 2
        delta_for_mideps = deltacomp(mideps, delta)
        if delta_for_mideps < delta:
            llim = llim
            rlim = mideps
        else:
            llim = mideps
            rlim = rlim
    return rlim

# ================/EXACT EMPIRICAL ANALYSIS WITH STEPS - SAMPLING EMPIRICAL/==============

#This a subroutine in the main algorithm.
def onestep(c, epsilon, eps0, pminusq):
    '''
    onestep computes the e^(epsilon)-divergence between p=alpha*Bin(c,0.5)+(1-alpha)*(Bin(c,1/2)+1) and q=alpha*(Bin(c,0.5)+1)+(1-alpha)*Bin(c,1/2), where alpha=e^(epsilon)/(1+e^(epsilon))
    if pminusq=True then computes D_(e^epsilon)(p|q), else computes D_(e^epsilon)(q|p)
    '''
    alpha = math.exp(eps0) / (math.exp(eps0) + 1)
    effeps = math.log(((math.exp(epsilon) + 1) * alpha - 1) / ((1 + math.exp(epsilon)) * alpha - math.exp(epsilon)))
    if pminusq == True:
        beta = 1 / (math.exp(effeps) + 1)
    else:
        beta = 1 / (math.exp(-effeps) + 1)
    cutoff = beta * (c + 1)
    pconditionedonc = (alpha * stats.binom.cdf(cutoff, c, 0.5) + (1 - alpha) * stats.binom.cdf(cutoff - 1, c, 0.5))
    qconditionedonc = ((1 - alpha) * stats.binom.cdf(cutoff, c, 0.5) + alpha * stats.binom.cdf(cutoff - 1, c, 0.5))
    if pminusq == True:
        return (pconditionedonc - math.exp(epsilon) * qconditionedonc)
    else:
        return ((1 - qconditionedonc) - math.exp(epsilon) * (1 - pconditionedonc))


def deltacomp(n, eps0, epsilon, deltaupper, step, upperbound = True):
    '''
    Let C=Bin(n-1, e^(-eps0)) and A=Bin(c,1/2) and B=Bin(c,1/2)+1 and alpha=e^(eps0)/(e^(eps0)+1)
    p samples from A w.p. alpha and B otherwise
    q samples from B w.p. alpha and A otherwise
    deltacomp attempts to find the smallest delta such P and Q are (epsilon,delta)-indistinguishable, or outputs deltaupper if P and Q are not (epsilon, deltaupper)-indistinguishable.
    If upperbound=True then this produces an upper bound on the true delta (except if it exceeds deltaupper), and if upperbound=False then it produces a lower bound.
    '''
    deltap = 0  # this keeps track of int max{0, p(x)-q(x)} dx
    deltaq = 0  # this keeps track of int max{0, q(x)-p(x)} dx
    probused = 0  # To increase efficiency, we're only to search over a subset of the c values.
    # This will keep track of what probability mass we have covered so far.

    p = math.exp(-eps0)
    expectation = (n-1)*p

    # Now, we are going to iterate over the n/2, n/2-step, n/2+step, n/2-2*steps, ...
    for B in range(1, int(np.ceil(n/step)), 1):
        for s in range(2):
            if s == 0:
                if B==1:
                    upperc = int(np.ceil(expectation+B*step))  # This is stepping up by "step".
                    lowerc = upperc - step
                else:
                    upperc = int(np.ceil(expectation + B * step))  # This is stepping up by "step".
                    lowerc = upperc - step + 1
                if lowerc>n-1:
                    inscope = False
                else:
                    inscope = True
                    upperc = min(upperc, n-1)
            if s == 1:
                lowerc = int(np.ceil(expectation-B*step))
                upperc = lowerc + step - 1
                if upperc<0:
                    inscope = False
                else:
                    inscope = True
                    lowerc = max(0, lowerc)

            if inscope == True:
                cdfinterval = stats.binom.cdf(upperc, n - 1, p) - stats.binom.cdf(lowerc, n - 1, p) + stats.binom.pmf(lowerc, n - 1, p)
            # This is the probability mass in the interval (in Bin(n-1, p))

                if max(deltap, deltaq) > deltaupper:
                    return deltaupper

                if 1 - probused < deltap and 1 - probused < deltaq:
                    if upperbound == True:
                        return max(deltap + 1 - probused, deltaq + 1 - probused)
                    else:
                        return max(deltap, deltaq)

                else:
                    deltap_upperc = onestep(upperc, epsilon, eps0, True)
                    deltap_lowerc = onestep(lowerc, epsilon, eps0, True)
                    deltaq_upperc = onestep(upperc, epsilon, eps0, False)
                    deltaq_lowerc = onestep(lowerc, epsilon, eps0, False)

                    if upperbound == True:
                        # compute the maximum contribution to delta in the segment.
                        # The max occurs at the end points of the interval due to monotonicity
                        deltapadd = max(deltap_upperc, deltap_lowerc)
                        deltaqadd = max(deltaq_upperc, deltaq_upperc)
                    else:
                        deltapadd = min(deltap_upperc, deltap_lowerc)
                        deltaqadd = min(deltaq_upperc, deltaq_lowerc)

                    deltap = deltap + cdfinterval * deltapadd
                    deltaq = deltaq + cdfinterval * deltaqadd

                probused = probused + cdfinterval  # updates the mass of C covered so far

    return max(deltap, deltaq)

def closedformanalysis(n, epsorig, delta):
    '''
    Theoretical computation the privacy guarantee of achieved by shuffling n eps0-DP local reports.
    '''
    if epsorig > math.log(n / (16 * math.log(4 / delta))):
        print("This is not a valid parameter regime for this analysis")
        return epsorig
    else:
        a = 8 * (math.exp(epsorig) * math.log(4 / delta)) ** (1 / 2) / (n) ** (1 / 2)
        c = 8 * math.exp(epsorig) / n
        e = math.log(1 + a + c)
        b = 1 - math.exp(-epsorig)
        d = (1 + math.exp(-epsorig - e))
        return math.log(1 + (b / d) * (a + c))

# #if UL=1 then produces upper bound, else produces lower bound.
def numericalanalysis(n, epsorig, delta, num_iterations, step, upperbound):
    '''
    Empirically computes the privacy guarantee of achieved by shuffling n eps0-DP local reports.
    num_iterations = number of steps of binary search, the larger this is, the more accurate the result
    If upperbound=True then this produces an upper bound on the true shuffled epsilon, and if upperbound=False then it produces a lower bound.
    '''
    # in order to speed things up a bit, we start the search for epsilon off at the theoretical upper bound.
    if epsorig < math.log(n / (16 * math.log(4 / delta))):
        # checks if this is a valid parameter regime for the theoretical analysis.
        # If yes, uses the theoretical upper bound as a starting point for binary search
        epsupper = closedformanalysis(n, epsorig, delta)
    else:
        epsupper = epsorig

    def deltacompinst(epsilon, delta):
        return deltacomp(n, epsorig, epsilon, delta, step, upperbound)

    return binarysearch(deltacompinst, delta, num_iterations, epsupper)

@cache
def shuffle_epsilon_analytic(sigma: float,
                             delta: float,
                             num_steps: int,
                             num_selected: int,
                             num_epochs: int,
                             step: float = 100,
                             ) -> float:
    if num_epochs > 1 or num_selected > 1:
        raise ValueError('Shuffle method only supports num_epochs=1 and num_selected=1')
    delta_split = 0.05
    det_eps = local_epsilon(sigma=sigma, delta=delta, num_selected=num_selected, num_epochs=num_epochs)
    local_delta = delta*delta_split/(2*num_steps*(np.exp(2)+1)*(1+np.exp(2)/2))
    local_epsilon_var = local_epsilon(sigma=sigma, delta=local_delta, num_selected=1, num_epochs=1)
    if local_epsilon_var is None or local_epsilon_var > 10:
        return det_eps
    epsilon = numericalanalysis(n=num_steps, epsorig=local_epsilon_var, delta=delta*(1-delta_split), num_iterations=num_epochs,
                                step=step, upperbound=True)
    for _ in range(5):
        local_delta = delta/(2*num_steps*(np.exp(epsilon)+1)*(1+np.exp(local_epsilon_var)/2))
        local_epsilon_var = local_epsilon(sigma, local_delta, num_selected=1, num_epochs=1)
        epsilon = epsilon = numericalanalysis(n=num_steps, epsorig=local_epsilon_var, delta=delta*(1-delta_split),
                                              num_iterations=num_epochs, step=step, upperbound=True)
        delta_bnd = delta*(1-delta_split)+local_delta*num_steps*(np.exp(epsilon)+1)*(1+np.exp(local_epsilon_var)/2)
        if delta_bnd < delta:
            break
    if epsilon > det_eps:
        return det_eps
    return epsilon

@cache
def shuffle_delta_analytic(sigma: float,
                           epsilon: float,
                           num_steps: int,
                           num_selected: int,
                           num_epochs: int,
                           step: float = 100,
                           ) -> float:
    if num_epochs > 1 or num_selected > 1:
        raise ValueError('Shuffle method only supports num_epochs=1 and num_selected=1')
    return bin_search(lambda delta: shuffle_epsilon_analytic(sigma=sigma, delta=delta, num_steps=num_steps, num_selected=num_selected,
                                                             num_epochs=num_epochs, step=step),
                      0, 1, epsilon, increasing=False)

### Random allocation

In [ ]:
# ================= Analytic ==================
@cache
def sampling_prob_from_sigma(sigma: float,
                             delta: float,
                             num_steps: int,
                             num_selected: int,
                             local_delta: float,
                             ) -> float:
    local_epsilon_val = local_epsilon(sigma=sigma, delta=local_delta, num_selected=num_selected, num_epochs=1)
    if local_epsilon_val is None:
        return 1.0
    gamma = np.cosh(local_epsilon_val)*np.sqrt(2*num_selected*np.log(num_selected/delta)/num_steps)
    if gamma > 0.99:
        return 1.0
    return np.clip(num_selected/(num_steps*(1.0-gamma)), 0, 1)

@cache
def allocation_epsilon_analytic(sigma: float,
                                delta: float,
                                num_steps: int,
                                num_selected: int,
                                num_epochs: int,
                                discretization: float = 1e-4,
                                ) -> float:
    local_delta_split = 0.99
    Poisson_delta_split = (1-local_delta_split)/2
    large_sampling_prob_delta_split = (1-local_delta_split)/2
    local_delta = delta*local_delta_split/(num_steps*num_epochs)
    Poisson_delta = delta*Poisson_delta_split
    large_sampling_prob_delta = delta*large_sampling_prob_delta_split/num_epochs
    sampling_prob = sampling_prob_from_sigma(sigma=sigma, delta=large_sampling_prob_delta, num_steps=num_steps,
                                             num_selected=num_selected, local_delta=local_delta)
    local_epsilon_val = local_epsilon(sigma=sigma, delta=delta, num_selected=num_selected, num_epochs=num_epochs)
    if sampling_prob > 0.99:
        return local_epsilon_val
    epsilon = poisson_epsilon_pld(sigma=sigma, delta=Poisson_delta, num_steps=num_steps, num_selected=num_selected,
                                  num_epochs=num_epochs, sampling_prob=sampling_prob, discretization=discretization)
    return min(epsilon, local_epsilon_val)

@cache
def allocation_delta_analytic(sigma: float,
                              epsilon: float,
                              num_steps: int,
                              num_selected: int,
                              num_epochs: int,
                              discretization: float = 1e-4,
                              ) -> float:
    return bin_search(lambda delta: allocation_epsilon_analytic(sigma=sigma, delta=delta, num_steps=num_steps,
                                                                num_selected=num_selected, num_epochs=num_epochs, discretization=discretization),
                      0, 1, epsilon, increasing=False)

# ==================== RDP ====================
@cache
def generate_partitions(n: int, max_size: int) -> List[List[Tuple[int, ...]]]:
    ''' Generate all integer partitions of [1, ..., n] with a maximum number of elements in the partition '''
    partitions = [[] for _ in range(n + 1)]
    partitions[0].append(())

    for i in range(1, n):
        partitions[i] = generate_partitions(n=i, max_size=max_size)
    for j in range(n, 0, -1):
        for p in partitions[n - j]:
            if (not p or j <= p[0]) and len(p) < max_size:  # Ensure descending order
                partitions[n].append((j,) + p)
    return partitions[n]

@jit(nopython=True)
def log_factorial(n: int) -> float:
    ''' Compute log(n!) '''
    if n <= 1:
        return 0.0
    return np.sum(np.log(np.arange(1, n + 1)))

@jit(nopython=True)
def log_factorial_range(n: int, m:int) -> float:
    ''' Compute log(n! / (n-m)!) '''
    if n <= 1:
        return 0.0
    return np.sum(np.log(np.arange(n - m + 1, n + 1)))

@jit(nopython=True)
def calc_partition_sum_square(arr: Tuple[int, ...]) -> float:
    ''' Compute the sum of squares of an array. e.g. (1, 2, 3) -> 1^2 + 2^2 + 3^2 '''
    result = 0.0
    for x in arr:
        result += x * x
    return result

@lru_cache(maxsize=None)
def calc_partition_sum_square_cached(arr: Tuple[int, ...]) -> float:
    ''' Compute the sum of squares of an array. e.g. (1, 2, 3) -> 1^2 + 2^2 + 3^2 '''
    return calc_partition_sum_square(arr=arr)

@jit(nopython=True)
def calc_log_multinomial(partition: Tuple[int, ...], n: int) -> float:
    ''' Compute the log of the multinomial of n over its partition'''
    log_prod_factorial = 0.0
    for p in partition:
        log_prod_factorial += log_factorial(n=p)

    return log_factorial(n=n) - log_prod_factorial

@lru_cache(maxsize=None)
def calc_log_multinomial_cached(partition: Tuple[int, ...], n: int) -> float:
    ''' Compute the log of the multinomial of n over its partition'''
    return calc_log_multinomial(partition=partition, n=n)

@jit(nopython=True)
def calc_counts_log_multinomial(partition: Tuple[int, ...], n: int) -> float:
    ''' Compute the counts of each unique integer in an array and calculate multinomial '''
    sum_partition = sum(partition)

    # Count frequencies
    counts = np.zeros(sum_partition + 1, dtype=np.int64)
    for x in partition:
        counts[x] += 1
    sum_counts = sum(counts)

    # Compute multinomial
    log_counts_factorial = 0.0
    for i in range(1, sum_partition + 1):
        if counts[i] > 0:
            log_counts_factorial += log_factorial(n=counts[i])

    return log_factorial_range(n=n, m=sum_counts) - log_counts_factorial

@lru_cache(maxsize=None)
def calc_counts_log_multinomial_cached(partition: Tuple[int, ...], n: int) -> float:
    ''' Compute the counts of each unique integer in an array and calculate multinomial '''
    return calc_counts_log_multinomial(partition=partition, n=n)

@cache
def compute_exp_term(partition: Tuple[int, ...], alpha: int, num_steps: int, sigma: float) -> float:
    ''' Compute the exponent term of the sum '''
    counts_log_multinomial = calc_counts_log_multinomial_cached(partition=partition, n=num_steps)
    partition_log_multinomial = calc_log_multinomial_cached(partition=partition, n=alpha)
    partition_sum_square = calc_partition_sum_square_cached(arr=partition) / (2 * sigma**2)
    return counts_log_multinomial + partition_log_multinomial + partition_sum_square

@cache
def allocation_rdp(alpha: int, sigma: float, num_steps: int) -> float:
    ''' Compute the RDP of the allocation mechanism '''
    partitions = generate_partitions(n=alpha, max_size=num_steps)
    exp_terms = [compute_exp_term(partition=partition, alpha=alpha, num_steps=num_steps, sigma=sigma) for partition in partitions]

    max_val = max(exp_terms)
    log_sum = np.log(sum(np.exp(term - max_val) for term in exp_terms))

    return (log_sum - alpha*(1/(2*sigma**2) + np.log(num_steps)) + max_val) / (alpha-1)

@cache
def allocation_epsilon_rdp(sigma: float,
                           delta: float,
                           num_steps: int,
                           num_selected: int,
                           num_epochs: int,
                           min_alpha: int = 2,
                           max_alpha: int = 50,
                           print_alpha: bool = False,
                           ) -> float:
    num_steps_per_round = np.ceil(num_steps/num_selected).astype(int)
    num_rounds = np.ceil(num_steps/num_steps_per_round).astype(int)
    alpha_rdp = allocation_rdp(min_alpha, sigma, num_steps_per_round)*num_rounds*num_epochs
    epsilon = alpha_rdp + math.log1p(-1/min_alpha) - math.log(delta * min_alpha)/(min_alpha-1)
    used_alpha = min_alpha
    alpha = int(min_alpha+1)
    surpased_rdp = False
    while alpha <= max_alpha and not surpased_rdp:
        alpha_rdp = allocation_rdp(alpha, sigma, num_steps_per_round)*num_rounds*num_epochs
        if alpha_rdp > epsilon:
            surpased_rdp = True
            break
        else:
            new_eps = alpha_rdp + math.log1p(-1/alpha) - math.log(delta * alpha)/(alpha-1)
            if new_eps < epsilon:
                epsilon = new_eps
                used_alpha = alpha
            alpha += 1
    if used_alpha == max_alpha:
        print(f'Potential alpha overflow! used alpha: {used_alpha} which is the maximal alpha')
        # return np.inf
    if used_alpha == min_alpha:
        print(f'Potential alpha underflow! used alpha: {used_alpha} which is the minimal alpha')
    if print_alpha:
        print(f'Used alpha: {used_alpha}')
    return epsilon

@cache
def allocation_delta_rdp(sigma: float,
                         epsilon: float,
                         num_steps: int,
                         num_selected: int,
                         num_epochs: int,
                         min_alpha: int = 2,
                         max_alpha: int = 50,
                         ) -> float:
    return bin_search(lambda delta: allocation_epsilon_rdp(sigma=sigma, delta=delta, num_steps=num_steps, num_selected=num_selected,
                                                           num_epochs=num_epochs, min_alpha=min_alpha, max_alpha=max_alpha),
                      0, 1, epsilon, increasing=False)

# ==================== RDP Inverse ====================
@cache
def allocation_rdp_inv(alpha: int, sigma: float, num_steps: int) -> float:
    ''' Compute the RDP of the allocation mechanism '''
    partitions = generate_partitions(n=alpha-1, max_size=num_steps)
    exp_terms = [compute_exp_term(partition=partition, alpha=alpha-1, num_steps=num_steps, sigma=sigma)
                 for partition in partitions]

    max_val = max(exp_terms)
    log_sum = np.log(sum(np.exp(term - max_val) for term in exp_terms))

    return (log_sum + max_val) / (alpha-1) - np.log(num_steps) + 1/(2*sigma**2)

@cache
def allocation_epsilon_rdp_inv(sigma: float,
                           delta: float,
                           num_steps: int,
                           num_selected: int,
                           num_epochs: int,
                           min_alpha: int = 2,
                           max_alpha: int = 50,
                           print_alpha: bool = False,
                           ) -> float:
    num_steps_per_round = np.ceil(num_steps/num_selected).astype(int)
    num_rounds = np.ceil(num_steps/num_steps_per_round).astype(int)
    alpha_rdp = allocation_rdp_inv(min_alpha, sigma, num_steps_per_round)*num_rounds*num_epochs
    epsilon = alpha_rdp + math.log1p(-1/min_alpha) - math.log(delta * min_alpha)/(min_alpha-1)
    used_alpha = min_alpha
    alpha = int(min_alpha+1)
    surpased_rdp = False
    while alpha <= max_alpha and not surpased_rdp:
        alpha_rdp = allocation_rdp_inv(alpha, sigma, num_steps_per_round)*num_rounds*num_epochs
        if alpha_rdp > epsilon:
            surpased_rdp = True
            break
        else:
            new_eps = alpha_rdp + math.log1p(-1/alpha) - math.log(delta * alpha)/(alpha-1)
            if new_eps < epsilon:
                epsilon = new_eps
                used_alpha = alpha
            alpha += 1
    if used_alpha == max_alpha:
        print(f'Potential alpha overflow! used alpha: {used_alpha} which is the maximal alpha')
        # return np.inf
    if used_alpha == min_alpha:
        print(f'Potential alpha underflow! used alpha: {used_alpha} which is the minimal alpha')
    if print_alpha:
        print(f'Used alpha: {used_alpha}')
    return epsilon

@cache
def allocation_delta_rdp_inv(sigma: float,
                         epsilon: float,
                         num_steps: int,
                         num_selected: int,
                         num_epochs: int,
                         min_alpha: int = 2,
                         max_alpha: int = 50,
                         ) -> float:
    return bin_search(lambda delta: allocation_epsilon_rdp_inv(sigma=sigma, delta=delta, num_steps=num_steps,
                                                               num_selected=num_selected, num_epochs=num_epochs,
                                                               min_alpha=min_alpha, max_alpha=max_alpha),
                      0, 1, epsilon, increasing=False)

# ==================== RDP Loose ====================

@cache
def allocation_rdp_loose(alpha: float, sigma: float, num_steps: int, num_selected: int) -> float:
    ''' Compute an upper bound on RDP of the allocation mechanism based on alpha=2 '''
    log_terms_arr = np.array([log_factorial_range(n=num_selected, m=i) - log_factorial(n=i)
                              + log_factorial_range(n=num_steps-num_selected, m=num_selected-i) - log_factorial(n=num_selected-i)
                              + i*alpha/(2*sigma**2) for i in range(num_selected+1)])
    max_log_term = np.max(log_terms_arr)
    return max_log_term + np.log(np.sum(np.exp(log_terms_arr - max_log_term))) - log_factorial_range(n=num_steps, m=num_selected) + log_factorial(n=num_selected)

@cache
def allocation_epsilon_rdp_loose(sigma: float,
                                 delta: float,
                                 num_steps: int,
                                 num_selected: int,
                                 num_epochs: int,
                                 print_alpha: bool = False,
                                 ) -> float:
    min_alpha = 2
    max_alpha = 500
    alpha_rdp = allocation_rdp_loose(min_alpha, sigma, num_steps, num_selected)*num_epochs
    epsilon = alpha_rdp + math.log1p(-1/min_alpha) - math.log(delta * min_alpha)/(min_alpha-1)
    used_alpha = min_alpha
    alpha = int(min_alpha+1)
    surpased_rdp = False
    while alpha <= max_alpha and not surpased_rdp:
        alpha_rdp = allocation_rdp_loose(alpha, sigma, num_steps, num_selected)*num_epochs
        if alpha_rdp > epsilon:
            surpased_rdp = True
            break
        else:
            new_eps = alpha_rdp + math.log1p(-1/alpha) - math.log(delta * alpha)/(alpha-1)
            if new_eps < epsilon:
                epsilon = new_eps
                used_alpha = alpha
            alpha += 1
    if used_alpha == max_alpha:
        print(f'Potential alpha overflow! used alpha: {used_alpha} which is the maximal alpha')
        # return np.inf
    if used_alpha == min_alpha:
        print(f'Potential alpha underflow! used alpha: {used_alpha} which is the minimal alpha')
    if print_alpha:
        print(f'Used alpha: {used_alpha}')
    return epsilon

@cache
def allocation_delta_rdp_loose(sigma: float,
                               epsilon: float,
                               num_steps: int,
                               num_selected: int,
                               num_epochs: int,
                               ) -> float:
    return bin_search(lambda delta: allocation_epsilon_rdp_loose(sigma=sigma, delta=delta, num_steps=num_steps,
                                                                 num_selected=num_selected, num_epochs=num_epochs),
                      0, 1, epsilon, increasing=False)

# ==================== Decomposition  ====================
@cache
def allocation_delta_decomposition(sigma: float,
                                   epsilon: float,
                                   num_steps: int,
                                   num_selected: int,
                                   num_epochs: int,
                                   discretization: float = 1e-4,
                                   ) -> float:
    num_steps_per_round = np.ceil(num_steps/num_selected).astype(int)
    num_rounds = np.ceil(num_steps/num_steps_per_round).astype(int)
    local_delta_val = local_delta(sigma, epsilon, num_epochs)
    lambda_val = 1 - (1-1.0/num_steps_per_round)**num_steps_per_round
    epsilon_new = np.log(1+lambda_val*(np.exp(epsilon)-1))
    delta_Poisson = poisson_delta_pld(sigma=sigma, epsilon=epsilon_new, num_steps=num_steps_per_round, num_selected=1,
                                      num_epochs=num_rounds*num_epochs, discretization=discretization)
    return min(local_delta_val, delta_Poisson / lambda_val)

@cache
def allocation_epsilon_decomposition(sigma: float,
                                     delta: float,
                                     num_steps: int,
                                     num_selected: int,
                                     num_epochs: int,
                                     discretization: float = 1e-4,
                                     ) -> float:
    num_steps_per_round = np.ceil(num_steps/num_selected).astype(int)
    num_rounds = np.ceil(num_steps/num_steps_per_round).astype(int)
    lambda_val = 1 - (1-1.0/num_steps_per_round)**num_steps_per_round
    delta_new = delta * lambda_val
    epsilon_Poisson = poisson_epsilon_pld(sigma=sigma, delta=delta_new, num_steps=num_steps_per_round, num_selected=1,
                                      num_epochs=num_rounds*num_epochs, discretization=discretization)
    factor = 1.0/lambda_val
    # use one of two identical formulas to avoid numerical instability
    if epsilon_Poisson < 1:
        amplified_epsilon = np.log(1+factor*(np.exp(epsilon_Poisson)-1))
    else:
        amplified_epsilon = epsilon_Poisson + np.log(factor + (1-factor)*np.exp(-epsilon_Poisson))
    local_epsilon_val = local_epsilon(sigma=sigma, delta=delta, num_selected=num_selected, num_epochs=num_epochs)
    return min(local_epsilon_val, amplified_epsilon)

# ==================== Monte Carlo ====================
def allocation_delta_monte_carlo_inner(sigma: float,
                                       epsilon: float,
                                       num_steps: int,
                                       num_experiments: float,
                                       ) -> float:
    means = np.zeros((num_steps, num_experiments))
    means[np.random.randint(num_steps, size=num_experiments), np.arange(num_experiments)] = 1
    samples = np.random.normal(means, sigma)
    loss = np.log(np.mean(np.exp(samples/sigma**2), axis=0)) - 1/(2*sigma**2)
    return np.clip(1 - np.exp(epsilon - loss), 0, None)

@cache
def allocation_delta_monte_carlo(sigma: float,
                                epsilon: float,
                                num_steps: int,
                                num_experiments: float,
                                num_epochs: int,
                                ) -> float:
    if num_epochs > 1:
        raise ValueError('Monte Carlo method only supports num_epochs=1')
    MAX_SIZE = 10_000_000
    if num_experiments*num_steps <= MAX_SIZE:
        samples = allocation_delta_monte_carlo_inner(sigma, epsilon, num_steps, num_experiments)
        mean = np.mean(samples)
        std = np.std(samples)
    else:
        experiments_per_round = MAX_SIZE // num_steps
        num_rounds = np.ceil(float(num_experiments) / experiments_per_round).astype(int)
        first_moment_total = 0
        sec_moment_total = 0
        for _ in range(num_rounds):
            samples = allocation_delta_monte_carlo_inner(sigma, epsilon, num_steps, experiments_per_round)
            mean = np.mean(samples)
            first_moment_total += mean
            sec_moment_total += np.var(samples) + mean**2
        mean = first_moment_total / num_rounds
        std = np.sqrt(sec_moment_total / num_rounds - mean**2)
    return mean

# def conditional_sampling(T: int, C: float, sigma: float):
#     '''
#     This function samples T elements from the the normal distribution with mean 0 and standard deviation sigma, conditioned on the event that the larges sample is larger than C
#     '''
#     # Sample x uniformly from [C, 1]
#     max_CDF = np.random.uniform(C, 1, T)
#     # Let y be the inverse CDF of x for distribution beta(T, 1)
#     max_val = stats.beta.ppf(max_CDF, 1, 1)
#     # sample a vector of T uniform random variables from the range [0, y]
#     arr_CDF = np.random.uniform(0, max_val, T)
#     # sample t uniformly from the range [T]
#     max_index = np.random.randint(T)
#     # set arr[t] to y
#     arr_CDF[max_index] = max_val
#     # set the output fo be the inverse CDF of arr for the distribution normal(0, sigma)
#     return stats.norm.ppf(arr_CDF, 0, sigma)

def conditional_sampling_Gaussian(dim: int, threshold: float, sigma: float, num_draws: int):
    '''
    Sample multiple draws from a multivariate Gaussian(0, sigma) of dimension dim
    conditioned on the maximum value per draw exceeding threshold C.
    '''
    # Sample max_CDF uniformly from [C, 1] for all draws
    max_CDF = np.random.uniform(threshold, 1, num_draws)
    # Get max_val using inverse CDF of beta(1, 1)
    max_val = stats.beta.ppf(max_CDF, 1, 1)
    # Create a matrix of random uniform values from [0, max_val] for each row
    # This is done by first sampling from uniform distribution over [0, 1] and then scaling by max_val
    arr_CDF = np.random.uniform(0, 1, (num_draws, dim)) * max_val.reshape(-1, 1)
    # Generate random indices for placing the max value in each row
    max_indices = np.random.randint(0, dim, num_draws)
    # Place max_val at the randomly chosen index for each row
    arr_CDF[:, max_indices] = max_val
    # Apply inverse normal CDF to transform to normal distribution
    return stats.norm.ppf(arr_CDF, 0, sigma)

def allocation_delta_monte_carlo_new(sigma: float,
                                       epsilon: float,
                                       num_steps: int,
                                       num_experiments: float,
                                       num_epochs: int,
                                       ) -> float:
    if num_epochs > 1:
        raise ValueError('Monte Carlo method only supports num_epochs=1')
    # Set a threshold that ensures privacy loss below epsilon for any value below it
    threshold = 0.5 + sigma**2*(epsilon - np.log(1-(np.exp(1/sigma**2)-1)/num_steps))
    # Sample elements from the normal distribution conditioned on the maximum value exceeding threshold
    samples = conditional_sampling_Gaussian(num_steps, threshold, sigma, num_experiments)
    # Add 1 to the first coordinate of each sample
    samples[:, 0] += 1
    # Compute the privacy loss
    loss = np.log(np.mean(np.exp(samples/sigma**2), axis=0)) - 1/(2*sigma**2)
    # Compute the divergence estimate
    div_est = np.mean(np.clip(1 - np.exp(epsilon - loss), 0, None))
    # Compute the probability that the maximal coordinate of a sample from a multivariate Gaussian(0, sigma) of dimension num_steps exceeds threshold, which is 1 - the probability that none of the coordinates exceed the threshold
    prob_exceed_threshold = 1 - (stats.norm.cdf(threshold/sigma))**num_steps
    # Rescale the divergence estimate by the probability of exceeding the threshold
    return div_est * prob_exceed_threshold

# Experiments code

In [ ]:
#======================= Variables =======================
EPSILON = 'epsilon'
DELTA = 'delta'
SIGMA = 'sigma'
NUM_STEPS = 'num_steps'
NUM_SELECTED = 'num_selected'
NUM_EPOCHS = 'num_epochs'
VARIABLES = [EPSILON, DELTA, SIGMA, NUM_STEPS, NUM_SELECTED, NUM_EPOCHS]

names_dict = {EPSILON: '$\\varepsilon$', DELTA: '$\\delta$', SIGMA: '$\\sigma$', NUM_STEPS: '$t$', NUM_SELECTED: '$k$',
              NUM_EPOCHS: '$E$'}

#===================== Configuration =====================
NUM_EXP = 'num_experiments'
DISCRETIZATION = 'discretization'
MIN_ALPHA = 'min_alpha'
MAX_ALPHA = 'max_alpha'
CONFIGS = [NUM_EXP, DISCRETIZATION, MIN_ALPHA, MAX_ALPHA]
ALPHA_ORDERS = 'alpha_orders'

# ======================= Schemes =======================
LOCAL = 'Local'
POISSON = 'Poisson'
ALLOCATION = 'allocation'
SHUFFLE = 'Shuffle'

colors_dict = {LOCAL: '#FF0000', POISSON: '#2BB22C', ALLOCATION: '#157DED', SHUFFLE: '#FF00FF'}

# ======================= Computation =======================
ANALYTIC = 'Analytic'
MONTE_CARLO = 'Monte Carlo'
PLD = 'PLD'
RDP = 'RDP'
DECOMPOSITION = 'Decomposition'

# ======================= Methods =======================
POISSON_PLD                 = f'{POISSON} ({PLD})'
POISSON_RDP                 = f'{POISSON} ({RDP})'
ALLOCATION_ANALYTIC         = f'{ALLOCATION} (Our - {ANALYTIC})'
ALLOCATION_RDP              = f'{ALLOCATION} (Our - {RDP})'
ALLOCATION_RDP_INV          = f'{ALLOCATION} (Our - {RDP} - Inv)'
ALLOCATION_RDP_LOOSE        = f'{ALLOCATION} (DCO25 - {RDP})'
ALLOCATION_DECOMPOSITION    = f'{ALLOCATION} (Our - {DECOMPOSITION})'
ALLOCATION_MONTE_CARLO      = f'{ALLOCATION} ({MONTE_CARLO})'
ALLOCATION_MONTE_CARLO_NEW  = f'{ALLOCATION} ({MONTE_CARLO} - New)'

@dataclass
class MethodFeatures:
    """
    Container for all features associated with a method.
    """
    name: str
    epsilon_calculator: Callable
    delta_calculator: Callable
    legend: str
    marker: str
    color: str

methods_dict = {
    LOCAL: MethodFeatures(
        name=LOCAL,
        epsilon_calculator=local_epsilon,
        delta_calculator=local_delta,
        legend='_{\\mathcal{L}}$ - ' + LOCAL,
        marker='*',
        color=colors_dict[LOCAL]
    ),
    POISSON_PLD: MethodFeatures(
        name=POISSON_PLD,
        epsilon_calculator=poisson_epsilon_pld,
        delta_calculator=poisson_delta_pld,
        legend='_{\\mathcal{P}}$ - ' + POISSON_PLD,
        marker='x',
        color=colors_dict[POISSON]
    ),
    POISSON_RDP: MethodFeatures(
        name=POISSON_RDP,
        epsilon_calculator=poisson_epsilon_rdp,
        delta_calculator=poisson_delta_rdp,
        legend='_{\\mathcal{P}}$ - ' + POISSON_RDP,
        marker='v',
        color=colors_dict[POISSON]
    ),
    SHUFFLE: MethodFeatures(
        name=SHUFFLE,
        epsilon_calculator=shuffle_epsilon_analytic,
        delta_calculator=shuffle_delta_analytic,
        legend='_{\\mathcal{S}}$ - ' + SHUFFLE,
        marker='p',
        color=colors_dict[SHUFFLE]
    ),
    ALLOCATION_ANALYTIC: MethodFeatures(
        name=ALLOCATION_ANALYTIC,
        epsilon_calculator=allocation_epsilon_analytic,
        delta_calculator=allocation_delta_analytic,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_ANALYTIC,
        marker='P',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_RDP: MethodFeatures(
        name=ALLOCATION_RDP,
        epsilon_calculator=allocation_epsilon_rdp,
        delta_calculator=allocation_delta_rdp,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_RDP,
        marker='^',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_RDP_INV: MethodFeatures(
        name=ALLOCATION_RDP,
        epsilon_calculator=allocation_epsilon_rdp_inv,
        delta_calculator=allocation_delta_rdp_inv,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_RDP + ' - inv',
        marker='*',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_RDP_LOOSE: MethodFeatures(
        name=ALLOCATION_RDP_LOOSE,
        epsilon_calculator=allocation_epsilon_rdp_loose,
        delta_calculator=allocation_delta_rdp_loose,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_RDP_LOOSE,
        marker='o',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_DECOMPOSITION: MethodFeatures(
        name=ALLOCATION_DECOMPOSITION,
        epsilon_calculator=allocation_epsilon_decomposition,
        delta_calculator=allocation_delta_decomposition,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_DECOMPOSITION,
        marker='X',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_MONTE_CARLO: MethodFeatures(
        name=ALLOCATION_MONTE_CARLO,
        epsilon_calculator=None,
        delta_calculator=allocation_delta_monte_carlo,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_MONTE_CARLO,
        marker='o',
        color=colors_dict[ALLOCATION]
    ),
    ALLOCATION_MONTE_CARLO_NEW: MethodFeatures(
        name=ALLOCATION_MONTE_CARLO_NEW,
        epsilon_calculator=None,
        delta_calculator=allocation_delta_monte_carlo_new,
        legend='_{\\mathcal{A}}$ - ' + ALLOCATION_MONTE_CARLO_NEW,
        marker='<',
        color=colors_dict[ALLOCATION]
    ),
}

### Supporting functions

In [ ]:
def get_features_for_methods(method_keys: List[str], feature_name: str) -> Dict[str, Any]:
    """
    Extract a specific feature for a list of methods using the global methods_dict.
    """
    if not all(key in methods_dict for key in method_keys):
        invalid_keys = [key for key in method_keys if key not in methods_dict]
        raise KeyError(f"Invalid method keys: {invalid_keys}")
    if not hasattr(methods_dict[method_keys[0]], feature_name):
        raise AttributeError(f"Invalid feature name: {feature_name}")
    return {key: getattr(methods_dict[key], feature_name) for key in method_keys}

def match_function_args(params_dict: Dict[str, Any],
                        config_dict: Dict[str, Any],
                        func: Callable,
                        x_var: str,
                        ) -> List[Dict[str, Any]]:
    params = inspect.signature(func).parameters
    args = {}
    for key in params_dict.keys():
        if key in params and key != x_var:
            args[key] = params_dict[key]
    for key in config_dict.keys():
        if key in params:
            args[key] = config_dict[key]
    args_arr = []
    for x in params_dict[x_var]:
        args_arr.append(args.copy())
        if x_var in params:
            args_arr[-1][x_var] = x
    return args_arr

def get_x_y_vars(params_dict: Dict[str, Any]) -> Tuple[str, str]:
    x_var = params_dict['x_var']
    if x_var not in params_dict.keys():
        raise ValueError(f"{x_var} was defined as the x-axis variable but does not appear in the params_dict.")
    y_var = params_dict['y_var']
    if y_var == x_var:
        raise ValueError(f"{x_var} was chosen as both the x-axis and y-axis variable.")
    return x_var, y_var

def get_main_var(params_dict: Dict[str, Any]) -> str:
    main_var = params_dict['main_var']
    if main_var not in params_dict.keys():
        raise ValueError(f"{main_var} was defined as the main variable but does not appear in the params_dict.")
    x_var = params_dict['x_var']
    if main_var == x_var:
        raise ValueError(f"{main_var} was chosen as both the main and x-axis variable.")
    y_var = params_dict['y_var']
    if main_var == y_var:
        raise ValueError(f"{main_var} was chosen as both the main and y-axis variable.")
    return main_var

def get_func_dict(methods: list[str],
                  y_var: str
                  ) -> Dict[str, Any]:
    if y_var == EPSILON:
        return get_features_for_methods(methods, 'epsilon_calculator')
    if y_var == DELTA:
        return get_features_for_methods(methods, 'delta_calculator')
    raise ValueError(f"Invalid y_var: {y_var}")

### Experiment code

In [ ]:
def calc_params_inner(params_dict: Dict[str, Any],
                      config_dict: Dict[str, Any],
                      methods: list[str],
                      )-> Dict[str, Any]:
    x_var, y_var = get_x_y_vars(params_dict)
    data = {'y data': {}}
    func_dict = get_func_dict(methods, y_var)
    for method in methods:
        start_time = time.time()
        func = func_dict[method]
        if func is None:
            raise ValueError(f"Method {method} does not have a valid function for {y_var}")
        args_arr = match_function_args(params_dict, config_dict, func, x_var)
        data['y data'][method] = np.array([func(**args) for args in args_arr])
        if data['y data'][method].ndim > 1:
            data['y data'][method + '- std'] = data['y data'][method][:,1]
            data['y data'][method] = data['y data'][method][:,0]
        end_time = time.time()
        print(f"Calculating {method} took {end_time - start_time:.3f} seconds")
    return data

def calc_params(params_dict: Dict[str, Any],
                config_dict: Dict[str, Any],
                methods: list[str],
                )-> Dict[str, Any]:
    x_var, y_var = get_x_y_vars(params_dict)
    data = calc_params_inner(params_dict, config_dict, methods)
    data['x name'] = names_dict[x_var]
    data['y name'] = names_dict[y_var]
    data['x data'] = params_dict[x_var]
    data['title'] = f"{names_dict[y_var]} as a function of {names_dict[x_var]} \n"
    for var in VARIABLES:
        if var != x_var and var != y_var:
            data[var] = params_dict[var]
            data['title'] += f"{names_dict[var]} = {params_dict[var]}, "
    return data

# Visualization

In [ ]:
def plot_comparison(data, log_x_axis = False, log_y_axis = False, format_x=lambda x, _: f'{x:.2f}', format_y=lambda x, _: f'{x:.2f}', figsize=(16, 9)):
    methods = list(data['y data'].keys())
    #remove keys that end with '- std'
    filtered_methods = [method for method in methods if not method.endswith('- std')]
    methods_data = data['y data']
    legend_map = get_features_for_methods(filtered_methods, 'legend')
    markers_map = get_features_for_methods(filtered_methods, 'marker')
    colors_map = get_features_for_methods(filtered_methods, 'color')
    legend_prefix = '$\\varepsilon' if data['y name'] == names_dict[EPSILON] else '$\\delta'
    plt.figure(figsize=figsize)
    for method in filtered_methods:
        plt.plot(data['x data'], methods_data[method], label=legend_prefix+legend_map[method], marker=markers_map[method], color=colors_map[method], linewidth=2.5, markersize=12, alpha=0.8)
        if method + '- std' in methods:
            plt.fill_between(data['x data'], np.clip(methods_data[method] - methods_data[method + '- std'], 0, 1),  np.clip(methods_data[method] + methods_data[method + '- std'], 0, 1), color=colors_map[method], alpha=0.1)
    plt.xlabel(data['x name'], fontsize=20)
    plt.ylabel(data['y name'], fontsize=20)
    if log_x_axis:
        plt.xscale('log')
        plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_x))
    else:
        plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_y))
    if log_y_axis:
        plt.yscale('log')
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xticks(data['x data'])
    plt.legend(fontsize=20)
    plt.show()

def plot_as_table(data):
    methods = list(data['y data'].keys())
    methods_data = data['y data']
    table = pd.DataFrame(methods_data, index=data['x data'])
    table.index.name = data['x name']
    table.columns = [method for method in methods]
    return table

def plot_combined_data(data, log_x_axis = False, log_y_axis = False, figsize=(16, 9)):
    methods = list(data['y data'].keys())
    if ALLOCATION_ANALYTIC in methods and ALLOCATION_RDP in methods and ALLOCATION_DECOMPOSITION in methods:
        min_allocation = np.min(np.array([data['y data'][ALLOCATION_ANALYTIC], data['y data'][ALLOCATION_RDP], data['y data'][ALLOCATION_DECOMPOSITION]]), axis=0)
    methods_data = data['y data']
    legend_map = get_features_for_methods(methods, 'legend')
    markers_map = get_features_for_methods(methods, 'marker')
    colors_map = get_features_for_methods(methods, 'color')
    legend_prefix = '$\\varepsilon' if data['y name'] == names_dict[EPSILON] else '$\\delta'
    plt.figure(figsize=figsize)
    for method in methods:
        linewidth = 1        if (method == ALLOCATION_DECOMPOSITION or method ==  ALLOCATION_RDP or method ==  ALLOCATION_ANALYTIC) else 2
        linestyle = 'dotted' if (method == ALLOCATION_DECOMPOSITION or method ==  ALLOCATION_RDP or method ==  ALLOCATION_ANALYTIC) else 'solid'
        plt.plot(data['x data'], methods_data[method], label=legend_prefix+legend_map[method], marker=markers_map[method], color=colors_map[method], linewidth=linewidth, linestyle=linestyle, markersize=10, alpha=0.8)
    if ALLOCATION_ANALYTIC in methods and ALLOCATION_RDP in methods and ALLOCATION_DECOMPOSITION in methods:
        plt.plot(data['x data'], min_allocation, label='_{\\mathcal{A}}$ - (Our - Combined)', color=colors_dict[ALLOCATION], linewidth=2, alpha=1)
    plt.xlabel(data['x name'], fontsize=20)
    plt.ylabel(data['y name'], fontsize=20)
    if log_x_axis:
        plt.xscale('log')
        plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))
    else:
        plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}'))
    if log_y_axis:
        plt.yscale('log')
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xticks(data['x data'])
    plt.legend(fontsize=20, loc='lower left', framealpha=0.)
    plt.show()

# Analysis

In [ ]:
alpha_arr = np.arange(2, 51)
sigma = 0.25
num_steps = 100
RDP_orig = np.array([allocation_rdp(alpha, sigma, num_steps) for alpha in alpha_arr])
RDP_inv = np.array([allocation_rdp_inv(alpha, sigma, num_steps) for alpha in alpha_arr])

#plot the RDP curves
plt.figure(figsize=(16, 9))
plt.plot(alpha_arr, RDP_orig, label='RDP', marker='o', color='blue', linewidth=2.5, markersize=12, alpha=0.8)
plt.plot(alpha_arr, RDP_inv, label='RDP - Inv', marker='x', color='red', linewidth=2.5, markersize=12, alpha=0.8)
plt.xlabel('Alpha', fontsize=20)
plt.ylabel('RDP', fontsize=20)
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.xticks(alpha_arr)
plt.legend(fontsize=20)
plt.title('RDP vs Alpha', fontsize=24)
plt.grid()
plt.show()

print(RDP_orig)
print(RDP_inv)

In [ ]:
params_dict_1 = {'x_var': SIGMA,
                 'y_var': EPSILON,
                 SIGMA: np.exp(np.linspace(np.log(0.2), np.log(5), 20)),
                 DELTA: 1e-10,
                 NUM_STEPS: 100_000,
                 NUM_SELECTED: 1,
                 NUM_EPOCHS: 1}

config_dict_1 = {DISCRETIZATION: 1e-4,
                 MIN_ALPHA: 2,
                 MAX_ALPHA: 60}

methods_list_1 = [LOCAL, POISSON_PLD, SHUFFLE, ALLOCATION_RDP, ALLOCATION_RDP_INV, ALLOCATION_ANALYTIC, ALLOCATION_DECOMPOSITION]

experiment_data_1 = calc_params(params_dict_1, config_dict_1, methods_list_1)

plot_combined_data(experiment_data_1, log_x_axis=True, log_y_axis=True)
plot_as_table(experiment_data_1)

In [ ]:
# params_dict_1 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.2), np.log(5), 20)),
#                  DELTA: 1e-10,
#                  NUM_STEPS: 500_000,
#                  NUM_SELECTED: 5,
#                  NUM_EPOCHS: 1}

# config_dict_1 = {DISCRETIZATION: 1e-4,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_1 = [LOCAL, POISSON_PLD, ALLOCATION_RDP, ALLOCATION_ANALYTIC]

# experiment_data_1 = calc_params(params_dict_1, config_dict_1, methods_list_1)

# plot_combined_data(experiment_data_1, log_x_axis=True, log_y_axis=True)
# plot_as_table(experiment_data_1)

In [ ]:
# params_dict_1 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.2), np.log(40), 20)),
#                  DELTA: 1e-10,
#                  NUM_STEPS: 100_000,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 1}

# config_dict_1 = {DISCRETIZATION: 1e-5,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60,
#                  ALPHA_ORDERS: ([1 + x / 10. for x in range(1, 100)] + list(range(11, 64)) + np.exp(np.linspace(np.log(64), np.log(2000), 20)).astype(int).tolist())}

# methods_list_1 = [LOCAL, POISSON_PLD, SHUFFLE, POISSON_RDP]

# experiment_data_1 = calc_params(params_dict_1, config_dict_1, methods_list_1)

# plot_combined_data(experiment_data_1, log_x_axis=True, log_y_axis=True)
# plot_as_table(experiment_data_1)

In [ ]:
params_dict_2 = {'x_var': NUM_EPOCHS,
                 'y_var': EPSILON,
                 SIGMA: 1,
                 DELTA: 1e-8,
                 NUM_STEPS: 10_000,
                 NUM_SELECTED: 1,
                 NUM_EPOCHS: np.exp(np.linspace(np.log(1), np.log(1001), 10)).astype(int)}

config_dict_2 = {DISCRETIZATION: 1e-4,
                 MIN_ALPHA: 2,
                 MAX_ALPHA: 60}

methods_list_2 = [POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_INV, POISSON_PLD, ALLOCATION_DECOMPOSITION]

experiment_data_2 = calc_params(params_dict_2, config_dict_2, methods_list_2)

plot_comparison(experiment_data_2, log_x_axis=True, log_y_axis=False, format_x=lambda x, _: x,)
plot_as_table(experiment_data_2)

In [ ]:
params_dict_3 = {'x_var': NUM_STEPS,
                 'y_var': DELTA,
                 SIGMA: 0.3,
                 EPSILON: 10,
                 NUM_STEPS: np.arange(25, 551, 50),
                 NUM_SELECTED: 1,
                 NUM_EPOCHS: 1}

config_dict_3 = {DISCRETIZATION: 1e-4,
                 NUM_EXP: 1_000_000,
                 MIN_ALPHA: 2,
                 MAX_ALPHA: 60}

methods_list_3 = [POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_INV, POISSON_PLD]

experiment_data_3 = calc_params(params_dict_3, config_dict_3, methods_list_3)

plot_comparison(experiment_data_3, log_x_axis=False, log_y_axis=True, format_x=lambda x, _: int(x),)
plot_as_table(experiment_data_3)

In [ ]:
params_dict_4 = {'x_var': NUM_SELECTED,
                 'y_var': EPSILON,
                 SIGMA: 1,
                 DELTA: 1e-6,
                 NUM_STEPS: 2**10,
                 NUM_SELECTED: 2**np.arange(0, 10),
                 NUM_EPOCHS: 1,}

config_dict_4 = {DISCRETIZATION: 1e-4,
                 MIN_ALPHA: 2,
                 MAX_ALPHA: 60}

methods_list_4 = [POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_INV, ALLOCATION_RDP_LOOSE]

experiment_data_4 = calc_params(params_dict_4, config_dict_4, methods_list_4)

plot_comparison(experiment_data_4, log_x_axis=True, log_y_axis=True, format_x=lambda x, _: x,)
plot_as_table(experiment_data_4)

In [ ]:
# params_dict_5 = {'x_var': NUM_EPOCHS,
#                  'y_var': DELTA,
#                  SIGMA: 2,
#                  EPSILON: 8,
#                  NUM_STEPS: 8,
#                  NUM_SELECTED: 4,
#                  NUM_EPOCHS: np.arange(4, 11, 1)}

# config_dict_5 = {DISCRETIZATION: 1e-5,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_5 = [POISSON_RDP, POISSON_PLD, ALLOCATION_RDP, ALLOCATION_RDP_LOOSE]

# experiment_data_5 = calc_params(params_dict_5, config_dict_5, methods_list_5)

# plot_comparison(experiment_data_5, log_x_axis=True, log_y_axis=False, format_x=lambda x, _: x,)
# plot_as_table(experiment_data_5)

In [ ]:
# params_dict_4 = {'x_var': NUM_SELECTED,
#                  'y_var': EPSILON,
#                  SIGMA: 1,
#                  DELTA: 1e-8,
#                  NUM_STEPS: 10_000,
#                  NUM_SELECTED: np.exp(np.linspace(np.log(1), np.log(100), 10)).astype(int),
#                  NUM_EPOCHS: 1,}

# config_dict_4 = {DISCRETIZATION: 1e-5,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_4 = [POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_LOOSE]

# experiment_data_4 = calc_params(params_dict_4, config_dict_4, methods_list_4)

# plot_comparison(experiment_data_4, log_x_axis=True, log_y_axis=False, format_x=lambda x, _: x,)
# plot_as_table(experiment_data_4)

In [ ]:
# params_dict_4 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.5), np.log(5), 10)),
#                  DELTA: 1e-6,
#                  NUM_STEPS: 8,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 10}

# config_dict_4 = {DISCRETIZATION: 1e-4,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_4 = [POISSON_PLD, POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_LOOSE, ALLOCATION_DECOMPOSITION]

# experiment_data_4 = calc_params(params_dict_4, config_dict_4, methods_list_4)

# plot_comparison(experiment_data_4, log_x_axis=True, log_y_axis=False)
# plot_as_table(experiment_data_4)

In [ ]:
# params_dict_5 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.5), np.log(5), 10)),
#                  DELTA: 1e-6,
#                  NUM_STEPS: 8,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 1024}

# config_dict_5 = {DISCRETIZATION: 1e-4,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_5 = [POISSON_PLD, POISSON_RDP, ALLOCATION_RDP, ALLOCATION_RDP_LOOSE, ALLOCATION_DECOMPOSITION]

# experiment_data_5 = calc_params(params_dict_5, config_dict_5, methods_list_5)

# plot_comparison(experiment_data_5, log_x_axis=True, log_y_axis=False)
# plot_as_table(experiment_data_1)

In [ ]:
# params_dict_6 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(10), np.log(20), 2)),
#                  DELTA: 1e-9,
#                  NUM_STEPS: 80,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 1}

# config_dict_6 = {DISCRETIZATION: 1e-4,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 60}

# methods_list_6 = [POISSON_PLD, POISSON_RDP, ALLOCATION_RDP, ALLOCATION_DECOMPOSITION]

# experiment_data_6 = calc_params(params_dict_6, config_dict_6, methods_list_6)

# plot_comparison(experiment_data_6, log_x_axis=True, log_y_axis=False)
# plot_as_table(experiment_data_6)

In [ ]:
# params_dict_7 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.5), np.log(5), 10)),
#                  DELTA: 1e-6,
#                  NUM_STEPS: 2**3,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 2**10}

# config_dict_7 = {DISCRETIZATION: 1e-4,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 10}

# methods_list_7 = [POISSON_RDP, ALLOCATION_RDP]

# experiment_data_7 = calc_params(params_dict_7, config_dict_7, methods_list_7)

# plot_combined_data(experiment_data_7, log_x_axis=True, log_y_axis=True)
# plot_as_table(experiment_data_7)

In [ ]:
# params_dict_8 = {'x_var': SIGMA,
#                  'y_var': EPSILON,
#                  SIGMA: np.exp(np.linspace(np.log(0.5), np.log(5), 10)),
#                  DELTA: 1e-6,
#                  NUM_STEPS: 2**3,
#                  NUM_SELECTED: 1,
#                  NUM_EPOCHS: 2**10}

# config_dict_8 = {DISCRETIZATION: 1e-4,
#                  NUM_EXP: 1_000_000,
#                  MIN_ALPHA: 2,
#                  MAX_ALPHA: 10,
#                  ALPHA_ORDERS: range(2, 11)}

# methods_list_8 = [POISSON_RDP, ALLOCATION_RDP]

# experiment_data_8 = calc_params(params_dict_8, config_dict_8, methods_list_8)

# plot_combined_data(experiment_data_8, log_x_axis=True, log_y_axis=True)
# plot_as_table(experiment_data_8)

In [ ]:
# sigma = 1
# num_steps = 1000
# min_alpha = 2
# max_alpha = 43
# alpha_arr = np.arange(min_alpha, max_alpha+1)
# exact_RDP = np.array([allocation_rdp(alpha, sigma, num_steps) for alpha in alpha_arr])
# approx_RDP = np.array([allocation_rdp(2, sigma*np.sqrt(2/alpha), num_steps) for alpha in alpha_arr])
# delta = 1e-6
# exact_eps = allocation_epsilon_rdp(sigma, delta, num_steps, 1, min_alpha=min_alpha, max_alpha=max_alpha)
# approx_eps = allocation_epsilon_rdp_loose(sigma, delta, num_steps, 1)
# print(f'Exact epsilon: {exact_eps}, Approx epsilon: {approx_eps}')

# plt.figure(figsize=(16, 9))
# plt.plot(alpha_arr, exact_RDP, label='Exact RDP', marker='o', color='blue', linewidth=2.5, markersize=12, alpha=0.8)
# plt.plot(alpha_arr, approx_RDP, label='Approx RDP', marker='x', color='red', linewidth=2.5, markersize=12, alpha=0.8)
# plt.xlabel('Alpha', fontsize=20)
# plt.ylabel('RDP', fontsize=20)
# plt.tick_params(axis='both', which='major', labelsize=12)
# plt.xticks(alpha_arr)
# plt.legend(fontsize=20)
# plt.show()

In [ ]:
# sigma = 2
# num_steps = 1000
# min_alpha = 2
# max_alpha = 43
# alpha_arr = np.arange(min_alpha, max_alpha+1)
# exact_RDP = np.array([allocation_rdp(alpha, sigma, num_steps) for alpha in alpha_arr])
# approx_RDP = np.array([allocation_rdp(2, sigma*np.sqrt(2/alpha), num_steps) for alpha in alpha_arr])
# delta = 1e-6
# exact_eps = allocation_epsilon_rdp(sigma, delta, num_steps, 1, min_alpha=min_alpha, max_alpha=max_alpha)
# approx_eps = allocation_epsilon_rdp_loose(sigma, delta, num_steps, 1)
# print(f'Exact epsilon: {exact_eps}, Approx epsilon: {approx_eps}')

# plt.figure(figsize=(16, 9))
# plt.plot(alpha_arr, exact_RDP, label='Exact RDP', marker='o', color='blue', linewidth=2.5, markersize=12, alpha=0.8)
# plt.plot(alpha_arr, approx_RDP, label='Approx RDP', marker='x', color='red', linewidth=2.5, markersize=12, alpha=0.8)
# plt.xlabel('Alpha', fontsize=20)
# plt.ylabel('RDP', fontsize=20)
# plt.tick_params(axis='both', which='major', labelsize=12)
# plt.xticks(alpha_arr)
# plt.legend(fontsize=20)
# plt.show()

In [ ]:
# sigma_arr = np.exp(np.linspace(np.log(0.5), np.log(5), 20))
# num_steps = 1000
# alpha = 30
# exact_RDP = np.array([allocation_rdp(alpha, sigma, num_steps) for sigma in sigma_arr])
# approx_RDP = np.array([allocation_rdp(2, sigma*np.sqrt(2/alpha), num_steps) for sigma in sigma_arr])

# plt.figure(figsize=(16, 9))
# plt.plot(sigma_arr, exact_RDP, label='Exact RDP', marker='o', color='blue', linewidth=2.5, markersize=12, alpha=0.8)
# plt.plot(sigma_arr, approx_RDP, label='Approx RDP', marker='x', color='red', linewidth=2.5, markersize=12, alpha=0.8)
# plt.xlabel('Sigma', fontsize=20)
# plt.ylabel('RDP', fontsize=20)
# plt.tick_params(axis='both', which='major', labelsize=12)
# plt.xticks(sigma_arr)
# plt.legend(fontsize=20)
# plt.yscale('log')
# plt.xscale('log')
# plt.show()


In [ ]:
# sigma = 2
# num_steps_arr = np.exp(np.linspace(np.log(10), np.log(100000), 20))
# alpha = 30
# exact_RDP = np.array([allocation_rdp(alpha, sigma, num_steps) for num_steps in num_steps_arr])
# approx_RDP = np.array([allocation_rdp(2, sigma*np.sqrt(2/alpha), num_steps) for num_steps in num_steps_arr])

# plt.figure(figsize=(16, 9))
# plt.plot(num_steps_arr, exact_RDP, label='Exact RDP', marker='o', color='blue', linewidth=2.5, markersize=12, alpha=0.8)
# plt.plot(num_steps_arr, approx_RDP, label='Approx RDP', marker='x', color='red', linewidth=2.5, markersize=12, alpha=0.8)
# plt.xlabel('Number of Steps', fontsize=20)
# plt.ylabel('RDP', fontsize=20)
# plt.tick_params(axis='both', which='major', labelsize=12)
# plt.xticks(num_steps_arr)
# plt.legend(fontsize=20)
# plt.yscale('log')
# plt.xscale('log')
# plt.show()
